In [47]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, InputLayer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder







In [48]:
# Load the dataset
file_path = '../dataset/train.csv'
df = pd.read_csv(file_path)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [49]:

# Preprocess the data
#df['comment_text'] = df['comment_text'].astype(str).apply(clean)  # Apply the cleaning function


In [50]:
# Assume binary classification: toxic vs. non-toxic
df['toxic'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)



In [51]:
# Tokenize the text
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['comment_text'])
sequences = tokenizer.texts_to_sequences(df['comment_text'])
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')


In [52]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['toxic'])

In [53]:

# Split the data
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)


In [58]:
# Build the model
model = Sequential([
    InputLayer(shape=(None,)),
    Embedding(input_dim=20000, output_dim=128, input_length=200),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [59]:

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [60]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, None, 128)           │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ (None, None, 128)           │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, None, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,701,089 (10.30 MB)

 Trainable params: 2,701,089 (10.30 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))




Epoch 1/5
3491/3491 ━━━━━━━━━━━━━━━━━━━━ 806s 229ms/step - accuracy: 0.9345 - loss: 0.2035 - val_accuracy: 0.9603 - val_loss: 0.1059
Epoch 2/5
3491/3491 ━━━━━━━━━━━━━━━━━━━━ 774s 222ms/step - accuracy: 0.9626 - loss: 0.1007 - val_accuracy: 0.9643 - val_loss: 0.1036
Epoch 3/5
3491/3491 ━━━━━━━━━━━━━━━━━━━━ 789s 226ms/step - accuracy: 0.9700 - loss: 0.0812 - val_accuracy: 0.9634 - val_loss: 0.1137
Epoch 4/5
3491/3491 ━━━━━━━━━━━━━━━━━━━━ 894s 256ms/step - accuracy: 0.9751 - loss: 0.0636 - val_accuracy: 0.9616 - val_loss: 0.1349
Epoch 5/5
3491/3491 ━━━━━━━━━━━━━━━━━━━━ 965s 276ms/step - accuracy: 0.9820 - loss: 0.0481 - val_accuracy: 0.9556 - val_loss: 0.1409


In [62]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

1496/1496 ━━━━━━━━━━━━━━━━━━━━ 112s 75ms/step - accuracy: 0.9564 - loss: 0.1386
Validation Loss: 0.14088128507137299
Validation Accuracy: 0.955631673336029


ValueError: Invalid dtype: str384

In [30]:
predict


array([[0.00569197],
       [0.00882977],
       [0.00888998],
       [0.00668387],
       [0.00668387],
       [0.09648935],
       [0.0129296 ],
       [0.14636339],
       [0.01154426],
       [0.00583188],
       [0.01140293],
       [0.01577565]], dtype=float32)

In [63]:

model.save('model.keras')

In [ ]:
# Load the dataset
import pandas as pd
import tensorflow as tf
file_path = '../dataset/train.csv'
df = pd.read_csv(file_path)

In [ ]:


# Load the trained model
model = tf.keras.models.load_model('model.keras')

# Load and fit the tokenizer
# Note: Make sure to use the same tokenizer parameters and fitting method used during training.
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
# Assuming 'comment_text' column contains the text data
tokenizer.fit_on_texts(df['comment_text'])  # Ensure 'df' is the same dataframe or load it again if needed

def preprocess_text(text):
    # Tokenize and pad the input text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=200, padding='post', truncating='post')
    return padded_sequence

def predict_text(text):
    # Preprocess the text
    padded_sequence = preprocess_text(text)
    # Predict using the loaded model
    prediction = model.predict(padded_sequence)
    print("preduition: ", prediction)
    # Interpret the prediction (assuming binary classification: 0 = non-toxic, 1 = toxic)
    if prediction >= 0.5:
        return "Toxic"
    else:
        return "Non-toxic"




In [ ]:
# Example usage
text = "aye you fresh air? cause..up dude bag"
result = predict_text(text)
print(f'The text "{text}" is classified as: {result}')